In [5]:
import os
import json
import ipywidgets as widgets
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

data_folder = '../ift6758/data/storage/dump/'
rink_image_path = '../figures/nhl_rink.png'

rink_width = 800
rink_height = 400

rink_x_min, rink_x_max = -100, 100
rink_y_min, rink_y_max = -42.5, 42.5

def load_season_data(season):
    file_path = os.path.join(data_folder, f"{season}.json")
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def filter_games_by_type(data, season_type):
    """
    Filters the game data by regular season ('02') or playoffs ('03').
    It checks the 5th and 6th characters of the game ID to determine the game type.
    """
    filtered_games = [game for game in data if str(game['id'])[4:6] == season_type]
    return filtered_games

def display_event_info(game_data, event_index):
    event_data = game_data['plays'][event_index]
    event_id = event_data['eventId']
    period = event_data['periodDescriptor']['number']
    time_in_period = event_data['timeInPeriod']
    event_type = event_data['typeDescKey']
    
    with event_output:
        clear_output(wait=True)
        print(f"Event ID: {event_id}")
        print(f"Period: {period}")
        print(f"Time in Period: {time_in_period}")
        print(f"Event Type: {event_type}")
        
        if 'details' in event_data:
            details = event_data['details']
            if 'xCoord' in details and 'yCoord' in details:
                print(f"Event Position: x={details['xCoord']}, y={details['yCoord']}")
                transformed_x, transformed_y = transform_coordinates(details['xCoord'], details['yCoord'])
                display_rink_image(transformed_x, transformed_y)
            else:
                display_rink_image()

            if 'reason' in details:
                print(f"Reason: {details['reason']}")
            if 'winningPlayerId' in details:
                print(f"Winning Player ID: {details['winningPlayerId']}")
            if 'losingPlayerId' in details:
                print(f"Losing Player ID: {details['losingPlayerId']}")
            if 'shootingPlayerId' in details:
                print(f"Shooting Player ID: {details['shootingPlayerId']}")
            if 'goalieInNetId' in details:
                print(f"Goalie in Net ID: {details['goalieInNetId']}")
            if 'hittingPlayerId' in details:
                print(f"Hitting Player ID: {details['hittingPlayerId']}")
            if 'hitteePlayerId' in details:
                print(f"Hittee Player ID: {details['hitteePlayerId']}")
            if 'blockingPlayerId' in details:
                print(f"Blocking Player ID: {details['blockingPlayerId']}")
        else:
            display_rink_image()

        print("\nJSON:")
        print(json.dumps(event_data, indent=4)) 

def display_game_info(season, game_index):
    data = load_season_data(season)
    season_type = season_type_selector.value
    filtered_game_data = filter_games_by_type(data, season_type)
    
    game_data = filtered_game_data[game_index]
    
    with match_output:
        clear_output(wait=True)
        print(f"Game ID: {game_data['id']}")
        print(f"Date: {game_data['gameDate']}")
        print(f"Home Team: {game_data['homeTeam']['name']['default']} (Score: {game_data['homeTeam']['score']})")
        print(f"Away Team: {game_data['awayTeam']['name']['default']} (Score: {game_data['awayTeam']['score']})")
        print(f"Venue: {game_data['venue']['default']} - {game_data['venueLocation']['default']}")
        print(f"Start Time (UTC): {game_data['startTimeUTC']}")
    
    event_slider.max = len(game_data['plays']) - 1
    event_slider.value = 0
    
    def update_event_output(*args):
        display_event_info(game_data, event_slider.value)
    
    event_slider.observe(update_event_output, names='value')
    
    with slider_output:
        clear_output(wait=True)
        display(event_slider)

def transform_coordinates(x, y):
    transformed_x = ((x - rink_x_min) / (rink_x_max - rink_x_min)) * rink_width
    transformed_y = rink_height - ((y - rink_y_min) / (rink_y_max - rink_y_min) * rink_height)
    return transformed_x, transformed_y

def display_rink_image(xCoord=None, yCoord=None):
    fig, ax = plt.subplots(figsize=(8, 4))
    img = mpimg.imread(rink_image_path)
    
    ax.imshow(img, extent=[0, rink_width, 0, rink_height])

    if xCoord is not None and yCoord is not None:
        ax.plot(xCoord, yCoord, 'go', markersize=8, label="Event Position")
        ax.legend()

    ax.set_xlim(0, rink_width)  
    ax.set_ylim(rink_height, 0) 

    ax.axis('off')
    plt.show()

season_selector = widgets.Dropdown(
    options=[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023], 
    description='Saison:'
)

season_type_selector = widgets.Dropdown(
    options={'All': 'all', 'Regular Season': '02', 'Playoffs': '03'},
    description='Season Type:'
)

game_slider = widgets.IntSlider(
    min=0,
    max=100,
    step=1,
    description='Match:'
)

event_slider = widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='Event:'
)

def update_game_slider(*args):
    season = season_selector.value
    season_type = season_type_selector.value
    data = load_season_data(season)
    
    filtered_data = filter_games_by_type(data, season_type) if season_type != 'all' else data
    game_slider.max = len(filtered_data) - 1
    game_slider.value = 0

def update_game_output(*args):
    display_game_info(season_selector.value, game_slider.value)

season_selector.observe(update_game_slider, names='value')
season_type_selector.observe(update_game_slider, names='value')
game_slider.observe(update_game_output, names='value')

match_output = widgets.Output()
slider_output = widgets.Output()
event_output = widgets.Output()

display(season_selector, season_type_selector, game_slider, match_output, slider_output, event_output)

update_game_slider()


Dropdown(description='Saison:', options=(2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023), value=2016)

Dropdown(description='Season Type:', options={'All': 'all', 'Regular Season': '02', 'Playoffs': '03'}, value='…

IntSlider(value=0, description='Match:')

Output()

Output()

Output()